In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [8]:
df = pd.read_csv('netflix_titles.csv')
df.head()


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


This section performs data processing and exploratory data analysis (EDA) on the Netflix dataset. I start by loading the data and displaying basic information, including the schema and the first few rows. The data is filtered to include only 'Movies' and 'TV Shows', and missing values in the 'director', 'cast', and 'country' columns are filled with 'Unknown'. I also convert the 'release_year' to integer format and extract the duration in minutes for movies. Duplicate entries are removed, and basic statistics, such as the number of rows, columns, and null values per column, are displayed. Finally, I aggregate the data to show the count of TV Shows vs Movies, the top 10 countries with the most content, and the distribution of releases by year and type.


In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_extract

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Netflix EDA") \
    .getOrCreate()

# Load dataset
df = spark.read.csv('netflix_titles.csv', header=True, inferSchema=True)

df.printSchema()
df.show(5)
valid_types = ['Movie', 'TV Show']
df = df.filter(df['type'].isin(valid_types))
df = df.fillna({'director': 'Unknown', 'cast': 'Unknown', 'country': 'Unknown'})
df = df.filter(df['type'].isNotNull() & df['title'].isNotNull())

df = df.withColumn('release_year', df['release_year'].cast('int'))

df = df.withColumn('duration_minutes', regexp_extract('duration', r'(\d+)', 1).cast('int'))

df = df.dropDuplicates(['show_id'])
print(f"Number of rows: {df.count()}")
print(f"Number of columns: {len(df.columns)}")
for col in df.columns:
    null_count = df.filter(df[col].isNull()).count()
    print(f"Column '{col}' has {null_count} null values")

df.describe().show()
df.groupBy('type').count().show()
df.groupBy('country').count().orderBy('count', ascending=False).show(10)

df.groupBy('release_year', 'type').count().orderBy('release_year', ascending=True).show()


root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)

+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|       director|                cast|      country|        date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+-------------------

this part will fill the null values with suitable default values

In [9]:
from pyspark.sql.functions import col, lit
df = df.fillna({
    'director': 'Unknown',
    'cast': 'Unknown',
    'country': 'Unknown',
    'date_added': 'Not Available',
    'rating': 'Not Rated',
    'release_year': 'Unknown',
    'title': 'Unknown'
})


In this section, I perform further exploratory data analysis (EDA) on the dataset by counting unique values in specific columns, such as 'type', 'country', 'rating', and 'listed_in'. For each of these columns, I use the `distinct()` method to get the unique values and then count how many there are. Additionally, I find the most frequent value in the 'country' column by grouping the data by 'country' and ordering the results by the count of occurrences, allowing me to identify which countries have the most content in the dataset.

In [13]:
for col_name in ['type', 'country', 'rating', 'listed_in']:
    unique_values = df.select(col_name).distinct().count()
    print(f"Column '{col_name}' has {unique_values} unique values")

from pyspark.sql.functions import count, desc
df.groupBy('country').count().orderBy(desc('count')).show(5)


Column 'type' has 2 unique values
Column 'country' has 749 unique values
Column 'rating' has 18 unique values
Column 'listed_in' has 514 unique values
+--------------+-----+
|       country|count|
+--------------+-----+
| United States| 2818|
|         India|  972|
|       Unknown|  831|
|United Kingdom|  419|
|         Japan|  245|
+--------------+-----+
only showing top 5 rows



I convert the `date_added` column to a date type, filter out invalid dates, and extract the year into a new `year_added` column. Then, I group the data by year, count the occurrences, and display the results.


In [17]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

from pyspark.sql.functions import to_date, year, col

df = df.withColumn("date_added", to_date("date_added", "MMMM d, yyyy"))

# Filter out rows with invalid dates
df = df.filter(col("date_added").isNotNull())

# Now extract the year and create a new column
df = df.withColumn("year_added", year("date_added"))

# Group by 'year_added' and count occurrences
df.groupBy("year_added").count().orderBy("year_added").show()


+----------+-----+
|year_added|count|
+----------+-----+
|      2008|    2|
|      2009|    2|
|      2010|    1|
|      2011|   13|
|      2012|    3|
|      2013|   10|
|      2014|   23|
|      2015|   73|
|      2016|  418|
|      2017| 1164|
|      2018| 1625|
|      2019| 1999|
|      2020| 1878|
|      2021| 1498|
+----------+-----+



In [ ]:
here is a little more detailed count by genre and country

In [18]:
# Top genres
df.groupBy('listed_in').count().orderBy(desc('count')).show(5)

# Top countries
df.groupBy('country').count().orderBy(desc('count')).show(5)


+--------------------+-----+
|           listed_in|count|
+--------------------+-----+
|Dramas, Internati...|  362|
|       Documentaries|  359|
|     Stand-Up Comedy|  334|
|Comedies, Dramas,...|  274|
|Dramas, Independe...|  252|
+--------------------+-----+
only showing top 5 rows

+--------------+-----+
|       country|count|
+--------------+-----+
| United States| 2778|
|         India|  971|
|       Unknown|  827|
|United Kingdom|  403|
|         Japan|  241|
+--------------+-----+
only showing top 5 rows



I extract the numeric duration from the `duration` column (e.g., converting '90 min' to 90) using `regexp_extract` and create a new column, `duration_num`. Then, I calculate the average duration of movies by filtering the dataset for movies and grouping by the `type` column before displaying the result.

In [19]:
# Extract numeric duration for movies (convert '90 min' to 90)
from pyspark.sql.functions import regexp_extract
df = df.withColumn("duration_num", regexp_extract('duration', r'(\d+)', 1).cast('int'))

# Average duration of movies
df.filter(df.type == 'Movie').groupBy('type').avg('duration_num').show()


+-----+-----------------+
| type|avg(duration_num)|
+-----+-----------------+
|Movie|99.57718668407311|
+-----+-----------------+

